In [6]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df = pd.read_csv("dataset/US_Accidents_March23.csv")

NameError: name 'pd' is not defined

In [ ]:
df = df.drop(columns=['End_Lat', 'End_Lng'], errors='ignore')

In [ ]:
date_columns = ['Start_Time', 'End_Time', 'Weather_Timestamp']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [ ]:
df = df.dropna(subset=['Weather_Timestamp', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Direction'])

In [ ]:
weather_fill_columns = ['Wind_Chill(F)', 'Precipitation(in)', 'Wind_Speed(mph)']
for col in weather_fill_columns:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
if 'Country' in df.columns and df['Country'].nunique() == 1:
    df = df.drop(columns=['Country'])

In [ ]:
df['Start_Hour'] = df['Start_Time'].dt.hour
df['Start_Weekday'] = df['Start_Time'].dt.weekday
df['Start_Month'] = df['Start_Time'].dt.month
df['Start_Year'] = df['Start_Time'].dt.year

In [ ]:
df['Duration(min)'] = (df['End_Time'] - df['Start_Time']).dt.total_seconds() / 60

In [ ]:
cat_columns = ['Source', 'State', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

for col in cat_columns:
    df[col] = df[col].astype(str).str.strip().str.lower()

In [ ]:
bool_columns = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 
                'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 
                'Traffic_Signal', 'Turning_Loop']

df[bool_columns] = df[bool_columns].astype(int)

In [ ]:
num_columns = ['Temperature(F)', 'Wind_Speed(mph)', 'Precipitation(in)', 
               'Humidity(%)', 'Pressure(in)', 'Visibility(mi)']

for col in num_columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

Handling Outliers

In [ ]:
df[col] = np.clip(df[col], lower_bound, upper_bound)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_') 

In [ ]:
cleaned_file_path = "dataset/cleaned_us_accident_data.csv"
df.to_csv(cleaned_file_path, index=False)